In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [6]:
bio = pd.read_csv("../dataset/Agriculture_biologique_final.csv", sep ="\t")
env = pd.read_csv("../dataset/Impact environnemental_final.csv", sep="\t",low_memory=False)
qual = pd.read_csv("../dataset/Qualite nutritionnelle_final.csv", sep="\t",low_memory=False)

# Nettoyage

## Agriculture Biologique

In [8]:
bio.isnull().sum()

product_name      7359
brands_tags          0
serving_size    377744
est_bio              0
dtype: int64